In [1]:
import os
import requests
from dotenv import load_dotenv
from urllib.parse import unquote
import xml.etree.ElementTree as ET
import pandas as pd
import time
from datetime import datetime

In [ ]:
# 지점코드 목록
station_codes = [90,212,93,216,95,217,98,221,99,226,100,232,101,235,102,236,104,
238,105,239,106,243,108,244,112,245,114,24,115,248,119,251,121,252,127,253,129,254,130,255,131,257,133,258,
135,259,136,260,137,261,138,262,140,263,143,264,146,266,152,268,155,271,156,272,159,273,162,276,165,277,168,
278,169,279,170,281,172,83,174,284,177,285,184,288,185,289,188,294,189,95,192,201,202,203,211
]

url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
params = {
    'serviceKey': 'Dowtu',
    'pageNo': '1',
    'numOfRows': '100',
    'dataType': 'XML',
    'dataCd': 'ASOS',
    'dateCd': 'DAY',
    'startDt': '20160101',
    'endDt': '20241231'
}

# 데이터를 저장할 리스트
all_data = []

# 각 지점코드를 순차적으로 처리
for stn_id in station_codes:
    params['stnIds'] = str(stn_id)  # 현재 지점코드 설정
    
    # 첫 번째 페이지의 응답을 요청하여 totalCount를 확인
    response = requests.get(url, params=params)

    if response.status_code == 200:
        root = ET.fromstring(response.content)

        # 응답에서 totalCount 찾기
        total_count = None
        for elem in root.iter():
            if elem.tag == 'totalCount':
                total_count = int(elem.text)
                break

        if total_count is not None:
            print(f"Processing station {stn_id} with total count: {total_count}")
            # 전체 페이지 수 계산 (페이지당 10개 데이터)
            num_of_rows = int(params['numOfRows'])
            total_pages = (total_count // num_of_rows) + (1 if total_count % num_of_rows != 0 else 0)

            # 전체 페이지 수만큼 반복하여 데이터 가져오기
            for page in range(1, total_pages + 1):
                params['pageNo'] = str(page)  # 현재 페이지 번호 설정
                response = requests.get(url, params=params)

                if response.status_code == 200:
                    root = ET.fromstring(response.content)

                    # XML에서 데이터를 추출
                    data = []
                    for item in root.iter('item'):
                        row = {}
                        for child in item:
                            row[child.tag] = child.text
                        data.append(row)

                    all_data.extend(data)  # 데이터를 리스트에 추가
                else:
                    print(f"Failed to fetch data for page {page}")
        else:
            print(f"Total count not found for station {stn_id}.")
    else:
        print(f"Failed to fetch data for station {stn_id}, status code: {response.status_code}")

# 모든 데이터를 DataFrame으로 변환
df = pd.DataFrame(all_data)


Processing station 90 with total count: 3288
Processing station 212 with total count: 3288
Processing station 93 with total count: 3014
Processing station 216 with total count: 3288
Processing station 95 with total count: 3288
Processing station 217 with total count: 3285
Processing station 98 with total count: 3288
Processing station 221 with total count: 3288
Processing station 99 with total count: 3288
Processing station 226 with total count: 3288
Processing station 100 with total count: 3288
Processing station 232 with total count: 3288
Processing station 101 with total count: 3287
Processing station 235 with total count: 3288
Processing station 102 with total count: 3288
Processing station 236 with total count: 3288
Processing station 104 with total count: 3288
Processing station 238 with total count: 3288
Processing station 105 with total count: 3288
Processing station 239 with total count: 2042
Processing station 106 with total count: 3288
Processing station 243 with total count

In [4]:
df.to_csv('weather_full.csv', index=False, encoding='utf-8')

In [6]:
weat = pd.read_csv('weather_full.csv')

C:\Users\cj033\AppData\Local\Temp\ipykernel_11616\3969378167.py:1: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  weat = pd.read_csv('weather_full.csv')


In [7]:
weat.head()

,stnId,stnNm,tm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt,mi10MaxRn,mi10MaxRnHrmt,...,avgM05Te,avgM10Te,avgM15Te,avgM30Te,avgM50Te,sumLrgEv,sumSmlEv,n99Rn,iscs,sumFogDur
0,90,속초,2016-01-01,3.6,-2.3,3.0,7.6,1302.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90,속초,2016-01-02,8.4,5.0,23.0,11.5,1508.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90,속초,2016-01-03,6.7,2.4,1937.0,9.8,1032.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90,속초,2016-01-04,5.4,0.8,2350.0,9.0,1348.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,90,속초,2016-01-05,0.6,-2.6,2228.0,4.4,1337.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
